In [2]:
import tensorflow as tf
import tensorflow.keras as K

In [3]:
def upscale_layer(layer, upscale_factor):
    '''
    int 만큼 layer(tensor)를 업스케일 합니다.
    텐서 크기 : [group, height, width, channels]
    '''
    height, width = layer.get_shape()[1:3]
    size = (upscale_factor * height, upscale_factor * width)
    upscaled_layer = tf.image.resize_nearest_neighbor(layer, size)
    return upscale_layer

In [4]:
def smoothly_merge_last_layer(list_of_layers, alpha):
    '''
    임계값 알파를 기반으로 층을 부드럽게 합침.
    이 합수는 모든 층이 이미 RGB로 바뀌었다고 가정함.
    생성자를 위한 함수.
    list_of_layers : 크기 순서대로 정렬된 텐서 리스트
    alpha : (0,1) 사이의 실수
    '''
    last_fully_trained_layer = list_of_layers[-2]
    last_layer_upscaled = upscale_later(last_fully_trained_layer, 2)
    larger_native_layer = list_of_layers[-1]
    
    assert larger_native_layer.get_shape() == last_layer_upscaled.get_shape()
    
    new_layer = (1-alpha) * last_layer_upscaled + larger_native_layer * alpha
    
    return new_layer
    

In [5]:
def minibatch_std_layer(layer, group_size=4):
    '''
    층의 미니배치 표준편차를 계산함.
    층의 데이터 타입은 float32로 가정. 아니라면 타입 변환.
    '''
    group_size = K.backend.minimum(group_size, tf.shape(layer)[0])
    shape = list(K.int_shape(input))
    shape[0] = tf.shape(input)[0]
    minibatch = K.backend.reshape(layer, (group_size, -1, shape[1], shape[2], shape[3]))
    
    minibatch -= tf.reduce_mean(minibatch, axis=0, keepdims=True)
    minibatch = tf.reduce_mean(K.backend.square(minibatch), axis=0)
    minibatch = K.backend.square(minibatch + 1e-8)
    minibatch = tr.reduce_mean(minibatch, axis=[1,2,3], keepdims=True)
    
    minibatch = K.backend.tile(minibatch, [group_size, 1, shape[2], shape[3]])
    return K.backend.concatenate([layer, minibatch], axis=1)